In [2]:
import sqlite3

In [4]:
# set up the database
conn = sqlite3.connect('ChoreTracker.sqlite3')

In [5]:
cursor = conn.cursor()

In [26]:
r = cursor.execute(""" SELECT name FROM sqlite_schema WHERE type='table' ORDER BY name;  """).fetchall()

In [27]:
r

[('choreinstances',),
 ('choreresponsibilities',),
 ('chores',),
 ('credentials',),
 ('people',),
 ('roles',),
 ('sqlite_sequence',)]

In [12]:
# Setting up the data model

r = cursor.execute("""
create table if not exists chores (
    ChoreID integer primary key AUTOINCREMENT,
    name text, 
    schedule text, -- how frequently does this chore happen (in days)?
    start_date text, -- when is the first day that this chore applies?
    start_time text, -- what time of day does the chore window open?
    window text, -- how many hours before the chore needs to be finished? 
    repeats INTEGER -- boolean to say whether this chore sould repeat or not
    );
""")


In [38]:
reset = False
if reset:
    r = cursor.execute("""
    delete from roles;
    """)
    
r = cursor.execute("""
create table if not exists roles (
    RoleID integer primary key AUTOINCREMENT,
    RoleName text UNIQUE -- parent or child
    );
""")

if reset:
    for role in ['Parent', 'Child']:
        r = cursor.execute(f"""
            INSERT INTO roles (RoleName) VALUES ('{role}');
        """)

conn.commit()

In [39]:
roles = cursor.execute(""" select * from roles """).fetchall()
roles

[(2, 'Parent'), (3, 'Child')]

In [45]:
role_ids = dict([(dict(roles)[k], k) for k in dict(roles)])

In [51]:
if reset:
    r = cursor.execute(""" drop table people """)

In [49]:
# Keep track of individuals so chores can be assigned to them
r = cursor.execute("""
create table if not exists people (
    PersonID integer primary key AUTOINCREMENT,
    PersonName text, 
    RoleID INTEGER, -- whether person is parent or child
    CurrentBalance INTEGER -- balance in pence, display number/100 for proper currency
    );
""")


r = cursor.execute(f"""
    INSERT INTO people (PersonName, RoleID, CurrentBalance) VALUES ('Mike', {role_ids['Parent']}, 0);
""")
conn.commit()


In [55]:
people = cursor.execute(""" select * from people """).fetchall()
[c[0] for c in cursor.description], people

(['PersonID', 'PersonName', 'RoleID', 'CurrentBalance'], [(1, 'Mike', 2, 0)])

In [21]:

# We're going to have a daily scheduled process to calculate the day's chores, and pop them in this table
# so we can track whether repeated chores are carried out 
r = cursor.execute("""
create table if not exists choreinstances (
    ChoreInstanceID integer primary key,
    ChoreID INTEGER, 
    Completed INTEGER, -- Boolean
    CompletedBy INTEGER, -- this will store PersonID of whoever completed it
    Banked INTEGER -- has it been paid into the bank account?
    );
""")


In [22]:

# Who is responsible for each chore?
# More than one person can be responsible, so we have to be 
# able to track whether a multi-person chore has been completed and by whom
r = cursor.execute("""
create table if not exists choreresponsibilities (
    ResponsibilityID integer primary key,
    PersonID INTEGER, 
    ChoreID INTEGER
    );
""")


In [56]:
# What's the going rate for a chore these days?
r = cursor.execute("""
create table if not exists chorerates (
    RateID integer primary key,
    Rate INTEGER, -- amount in pence
    StartDate TEXT -- when did the rate take effect?
    );
""")

In [57]:
r = cursor.execute(f"""
    INSERT INTO chorerates (Rate, StartDate) VALUES (25, '2023-01-01');
""")
conn.commit()

In [62]:
import pandas as pd

In [65]:
rates = cursor.execute(""" select rate, date(StartDate) as StartDate from chorerates """).fetchall()
pd.DataFrame(rates, columns=[c[0] for c in cursor.description])

,Rate,StartDate
0,25,2023-01-01


In [66]:

# Need to have some way of storing the parent passcode
# and potentially change it.

r = cursor.execute("""
create table if not exists credentials (
    CredentialID integer primary key,
    RoleID INTEGER, -- need one or both of these two
    PersonID INTEGER, -- need one or both of these two
    Passcode TEXT -- this should possibly be encrypted...
    );
""")